In [1]:
import torch 
from torch import nn

In [2]:
test_input = torch.randn(4, 3, 5, 32, 32)

In [4]:
conv2d = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

In [3]:
[i.shape for i in torch.chunk(test_input, 4, dim=1)]

[torch.Size([4, 1, 5, 32, 32]),
 torch.Size([4, 1, 5, 32, 32]),
 torch.Size([4, 1, 5, 32, 32])]

In [9]:
[i.shape for i in torch.chunk(test_input, 3, dim=1)]

[torch.Size([4, 1, 5, 32, 32]),
 torch.Size([4, 1, 5, 32, 32]),
 torch.Size([4, 1, 5, 32, 32])]

In [11]:
conv = nn.Conv3d(3, 10, 3, padding=(0, 1, 1))

In [12]:
conv(test_input).shape

torch.Size([4, 10, 3, 32, 32])

In [ ]:
class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding, activation, frame_size):

        super(ConvLSTMCell, self).__init__()  

        if activation == "tanh":
            self.activation = torch.tanh 
        elif activation == "relu":
            self.activation = torch.relu
        
        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        self.conv = nn.Conv2d(
            in_channels=in_channels + out_channels, 
            out_channels=4 * out_channels, 
            kernel_size=kernel_size, 
            padding=padding)           

        # Initialize weights for Hadamard Products
        self.W_ci = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_co = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_cf = nn.Parameter(torch.Tensor(out_channels, *frame_size))

    def forward(self, X, H_prev, C_prev):

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)

        input_gate = torch.sigmoid(i_conv + self.W_ci * C_prev )
        forget_gate = torch.sigmoid(f_conv + self.W_cf * C_prev )

        # Current Cell output
        C = forget_gate*C_prev + input_gate * self.activation(C_conv)

        output_gate = torch.sigmoid(o_conv + self.W_co * C )

        # Current Hidden State
        H = output_gate * self.activation(C)

        return H, C

In [13]:
convlstm = nn.ConvLSTM(10, 20, 3, padding=1)

AttributeError: module 'torch.nn' has no attribute 'ConvLSTM'